# **Deep learning neural network from scratch**
By Gabriel Pišvejc (gabrielzelva@gmail.com)

In this notebook, I will create a deep neural network from scratch using linear algebra and calculus. The network will then be used to tell genuine bank notes from forged ones.

The dataset I will be using can be found here:

Lohweg, V. (2012). Banknote Authentication [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C55P57.

## **Information about the data**

The original author provides a good description of what the data actually is: 

> Data were extracted from images that were taken from genuine and forged banknote-like specimens.  For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.

Additionally, I should mention that the data contains 4 predictors: 
- Variance
- Skewness
- Curtosis
- Entropy

The target is of course a boolean variable which indicates whether a certain specimen is genuine or forged. 

Finally, since the dataset is not particularly large, I will be working with a seed to prevent bad luck ruining this demonstration. 

In [1]:
seed = 3091488470

# **Creating the neural network**

In this section, I will show the step by step process for creating the model

## **Data preprocessing**

First things first, beside actually loading the data, we will also need to preprocess it. That means:
- Shuffle it in order to prevent any ordering bias
- Map the predictors to a standard normal distribution
- Separate the predictors from the response
- Separate the training data from the testing data

In [2]:
# Loading the data
import pandas as pd

data = pd.read_csv("data_banknote_authentication.txt")

In [3]:
# Shuffling the data

data = data.sample(frac = 1,
                   random_state = seed,
                   replace = False,
                   ignore_index = True)

In [4]:
# Mapping the predictors to a standard normal distribution

data.iloc[:, :4] = data.iloc[:, :4].apply(
    lambda col: (col - col.mean()) / col.std()
)

In [5]:
# Separating the predictors from the response

predictors = data.iloc[:, :4]

response = data.iloc[:, 4:5]

In [6]:
# Separating the testing data from the training data
import numpy as np 

split_index = int(len(data) * 2/3)

training_predictors = predictors.iloc[:split_index, :]
training_predictors = np.array(training_predictors)

testing_predictors = predictors.iloc[split_index:, ]
testing_predictors = np.array(testing_predictors)


training_response = response.iloc[:split_index, :]
training_response = np.array(training_response)

testing_response = response.iloc[split_index:, :]
testing_response = np.array(testing_response)



del predictors, response, split_index

## **Declaring the neural network**

I want to create a neural network with the following characteristics: 
- 4 input nodes, one per each predictor
- 2 hidden layers, each containing 16 nodes and followed by the ReLU activation function
- 1 output node, followed by the logistic function to predict a binary outcome (the bank note is either forged or genuine)

We can see the desired network on the following diagram: 

<img src="Diagram.jpeg" width="400" height="400">

### **The maths behind the structure**

#### **The tensors**

To build this neural network, we will need to view each hidden layer as a matrix of weights and a vector of biases. Therefore, the first hidden layer can be expressed like so:

$
W_{FirstHidden} \begin{bmatrix}
w_{1.1} & w_{1.2} & \cdots & w_{1.16} \\
w_{2.1} & w_{2.2} & \cdots & w_{2.16} \\
w_{3.1} & w_{3.2} & \cdots & w_{3.16} \\
w_{4.1} & w_{4.2} & \cdots & w_{4.16}
\end{bmatrix}
$
$
B_{FirstHidden} = \begin{bmatrix}
b_1 \\
b_2 \\
\vdots \\
b_{16}
\end{bmatrix}
$

The second hidden layer like so: 

$
W_{SecondHidden} \begin{bmatrix}
w_{1.1} & w_{1.2} & \cdots & w_{1.16} \\
w_{2.1} & w_{2.2} & \cdots & w_{2.16} \\
\vdots & \vdots & \ddots & \vdots \\
w_{16.1} & w_{16.2} & \cdots & w_{16.16}
\end{bmatrix}
$
$
B_{SecondHidden} = \begin{bmatrix}
b_1 \\
b_2 \\
\vdots \\
b_{16}
\end{bmatrix}
$

And finally, the output layer can be expressed like a simple vector of the 16 weights converging to a single node and the remaining bias can be seen as a scalar. 

$
W_{Output} = \begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_{16}
\end{bmatrix}
$
$
B_{Output} = [b]
$

Of course, we will define all of these tensors in Python using NumPy:

In [7]:
# We shouldn't really define it just now, but for narrative purposes, I will leave the block here.
# However, we will need to redefine it later

np.random.seed(seed)

w_first_hidden = np.random.rand(16, 4)
b_first_hidden = np.random.rand(16, 1)

w_second_hidden = np.random.rand(16, 16)
b_second_hidden = np.random.rand(16, 1)

w_output = np.random.rand(1, 16)
b_output = np.random.rand(1, 1)


#### **Activation functions**

Needless to say, we also need the activation functions. As we have previously established, we will use ReLU for the two hidden layers and the logistic function for the output. 

$ \Large
ReLU(x) = \frac{x + |x|}{2}
$
$\Large
Logistic(x) = \frac{1}{1+e^{-x}}
$

We can define them in Python like so: 

In [8]:
def relu(x): 
    return np.maximum(x, 0) 
    # We could use the function literally: 
    # return (x + np.abs(x)) / 2
    # But np.maximum does exactly the same thing while being much more readeable, so, I am going with that. 
    
def logistic(x): 
    return 1 / (1 + np.exp(-x))

## **Making the structure work**

Now that we have created the individual blocks of our neural network, we also need to force them to operate like one. That is, we need to:

- Create a function for the forward propagation which will allow us to make predictions.
- Create a function for the backward propagation which will allow us to actually train the model. 

### **Forward propagation**

For a prediction to be made, the following needs to happen: 
1. The values of the 4 predictors need to be passed into the first hidden layer as a vector.
2. This will cause a matrix-vector multiplication between the entry vector and the weight matrix.
3. The layer will also sum the vector of biases to the now multiplied vector.
4. ReLU will be applied to the resulting vector.
5. Having finished with the first hidden layer, the new vector will be passed to the second one which will repeat the steps 2, 3 and 4, however, using its own weight matrix and bias vector.
6. Once the data is out of the hidden layers, it will converge to the output node. This will force the hidden layers result to be multiplied with the output weigh vector, producing a single scalar result.
7. The output bias will be summed to it.
8. The logistic function will be applied, mapping the result between 0 and 1. This is our result, since rounding this number will give us a prediction to whether a particular banknote was forged or not. 

Of course, we will do all of this via a function:

In [9]:
def forward_propagation(input_vector):
    
    first_algebra = w_first_hidden @ input_vector + b_first_hidden
    first_function = relu(first_algebra)
    
    second_algebra = w_second_hidden @ first_function + b_second_hidden
    second_function = relu(second_algebra)
    
    output_algebra = w_output @ second_function + b_output
    result = logistic(output_algebra)
    
    return first_algebra, first_function, second_algebra, second_function, output_algebra, result
    # We will also save the temporary states of the process for future use during training

### **Backward propagation**

Machine learning usually boils down to optimizing a loss function. However, unlike with linear regression, it can be a bit more tricky to take the derivative with respect to a certain variable, since, as we might suspect from seeing the forward propagation function, the loss function features some deep nesting. However, we can untangle this using the chain rule. 

We will look at how it is done in a moment, however, we first need to define our loss function. In this particular case, we will use the square error. I should also mention that in order to make the connection to the forward propagation easier, I will keep using the variable names from the function defined above. 

$ \Large Loss = (result - real\_value)^2 $ 

This formula, while being technically correct, is deceptively simple. The  reality is that "result" is actually nesting a series of functions that look like so: 

$
result = Logistic(output\_algebra)
$

$
output\_algebra = w\_output \cdot second\_function + b\_output
$

$
second\_function = ReLU(second\_algebra)
$

$
second\_algebra = w\_second\_hidden \cdot first\_function + b\_second\_hidden
$

$
first\_function = ReLU(first\_algebra)
$

$
first\_algebra = w\_first\_hidden \cdot input\_vector + b\_first\_hidden
$

Knowing this, we can use the chain rule to get all the derivatives we need. For example, if we were curious about the impact of the weights converging towards the output node, we could find their slope like so: 

$\Large
\frac{\delta \; loss}{\delta \; result} = 2 \cdot result - 2 \cdot real\_value
$

$\Large
\frac{\delta \; result}{\delta \; output\_algebra} = \frac{e^{-output\_algebra}}{(1+e^{-output\_algebra})^2}
$

$\Large
\frac{\delta \; output\_algebra}{\delta \; w\_output} = second\_function
$

$\Large
\therefore
$

$\Large
\frac{\delta \; loss}{\delta \; w\_output} = 
\frac{\delta \; loss}{\delta \; result} \cdot
\frac{\delta \; result}{\delta \; output\_algebra}\cdot
\frac{\delta \; output\_algebra}{\delta \; w\_output} = 
$

$\Large
(2 \cdot result - 2 \cdot real\_value) \cdot
(\frac{e^{-output\_algebra}}{(1+e^{-output\_algebra})^2}) \cdot
second\_function
$


While we obviously want to optimise each and every parameter inside our network, you might have noticed that the maths get really chaotic really quickly when done by hand. Therefore, we will find the derivatives we need using Python. 

First things first, we will find all of the individual derivatives:

In [10]:
import sympy as sp

# sympy doesnt't work with numpy's functions, therefore so for the defivatives purpuse,
# I will create a sympy version of ReLU and the logistic function

def sp_relu(x): 
    return sp.Max(x, 0)

def sp_logistic(x): 
    return 1 / (1 + sp.exp(-x)) 

# Define all the functions and take their derivatives

result, real_value, output_algebra, w_output, second_function, b_output, second_algebra, w_second_hidden, first_function, b_second_hidden, first_algebra, w_first_hidden, input_vector, b_first_hidden = \
    sp.symbols('result real_value output_algebra w_output second_function b_output second_algebra w_second_hidden first_function b_second_hidden first_algebra w_first_hidden input_vector b_first_hidden')

## Loss function
loss = (result - real_value)**2

d_loss__d_result = sp.diff(loss, result)
print("d_loss__d_result = ", d_loss__d_result)

## Output layer
result = sp_logistic(output_algebra)

d_result__d_output_algebra = sp.diff(result, output_algebra)
print("d_result__d_output_algebra = ", d_result__d_output_algebra)

output_algebra = w_output * second_function + b_output

d_output_algebra__d_w_output = sp.diff(output_algebra, w_output)
print("d_output_algebra__d_w_output = ", d_output_algebra__d_w_output)

d_output_algebra__d_b_output = sp.diff(output_algebra, b_output)
print("d_output_algebra__d_b_output = ", d_output_algebra__d_b_output)

d_output_algebra__d_second_function = sp.diff(output_algebra, second_function)
print("d_output_algebra__d_second_function = ", d_output_algebra__d_second_function)

## Second hidden layer
second_function = sp_relu(second_algebra)

d_second_function__d_second_algebra = sp.diff(second_function, second_algebra)
print("d_second_function__d_second_algebra = ", d_second_function__d_second_algebra)

second_algebra = w_second_hidden * first_function + b_second_hidden

d_second_algebra__d_w_second_hidden = sp.diff(second_algebra, w_second_hidden)
print("d_second_algebra__d_w_second_hidden = ", d_second_algebra__d_w_second_hidden)

d_second_algebra__d_b_second_hidden = sp.diff(second_algebra, b_second_hidden)
print("d_second_algebra__d_b_second_hidden = ", d_second_algebra__d_b_second_hidden)

d_second_algebra__d_first_function = sp.diff(second_algebra, first_function)
print("d_second_algebra__d_first_function = ", d_second_algebra__d_first_function)

## First hidden layer
first_function = sp_relu(first_algebra)

d_first_function__d_first_algebra = sp.diff(first_function, first_algebra)
print("d_first_function__d_first_algebra = ", d_first_function__d_first_algebra)

first_algebra = w_first_hidden * input_vector + b_first_hidden

d_first_algebra__d_w_first_hidden = sp.diff(first_algebra, w_first_hidden)
print("d_first_algebra__d_w_first_hidden = ", d_first_algebra__d_w_first_hidden)

d_first_algebra__d_b_first_hidden = sp.diff(first_algebra, b_first_hidden)
print("d_first_algebra__d_b_first_hidden = ", d_first_algebra__d_b_first_hidden)


d_loss__d_result =  -2*real_value + 2*result
d_result__d_output_algebra =  exp(-output_algebra)/(1 + exp(-output_algebra))**2
d_output_algebra__d_w_output =  second_function
d_output_algebra__d_b_output =  1
d_output_algebra__d_second_function =  w_output
d_second_function__d_second_algebra =  Heaviside(second_algebra)
d_second_algebra__d_w_second_hidden =  first_function
d_second_algebra__d_b_second_hidden =  1
d_second_algebra__d_first_function =  w_second_hidden
d_first_function__d_first_algebra =  Heaviside(first_algebra)
d_first_algebra__d_w_first_hidden =  input_vector
d_first_algebra__d_b_first_hidden =  1


We can use these and implement the chain rule as a function which will find the gradients of all of the matrices and vectors that can be used to optimise our network. This is what is called Backward propagation. 

In [11]:
# Before we continue, I will just redefine the linear algebra from before. The last code block messes it up. 

np.random.seed(seed)

w_first_hidden = np.random.rand(16, 4)
b_first_hidden = np.random.rand(16, 1)

w_second_hidden = np.random.rand(16, 16)
b_second_hidden = np.random.rand(16, 1)

w_output = np.random.rand(1, 16)
b_output = np.random.rand(1, 1)


In [12]:
def d_relu(x):
    return x > 0

def d_logistic(x):
    return np.exp(-x) / (1 + np.exp(-x)) ** 2

def backward_propagation(first_algebra, first_function, second_algebra, second_function, output_algebra, result, input_vector, real_value):
    # Output layer
    d_loss__d_result = 2 * (result - real_value)  # (1, 1)
    d_result__d_output_algebra = d_logistic(output_algebra)  # (1, 1)
    d_loss__d_output_algebra = d_loss__d_result * d_result__d_output_algebra  # (1, 1)

    d_loss__d_w_output = d_loss__d_output_algebra @ second_function.T  # (1, 16)
    d_loss__d_b_output = d_loss__d_output_algebra  # (1, 1)

    d_loss__d_second_function = w_output.T @ d_loss__d_output_algebra  # (16, 1)

    # Second hidden layer
    d_second_function__d_second_algebra = d_relu(second_algebra)  # (16, 1)
    d_loss__d_second_algebra = d_second_function__d_second_algebra * d_loss__d_second_function  # (16, 1)

    d_loss__d_w_second_hidden = d_loss__d_second_algebra @ first_function.T  # (16, 16)
    d_loss__d_b_second_hidden = d_loss__d_second_algebra  # (16, 1)

    # First hidden layer
    d_loss__d_first_function = w_second_hidden.T @ d_loss__d_second_algebra  # (16, 1)
    d_first_function__d_first_algebra = d_relu(first_algebra)  # (16, 1)
    d_loss__d_first_algebra = d_first_function__d_first_algebra * d_loss__d_first_function  # (16, 1)

    d_loss__d_w_first_hidden = d_loss__d_first_algebra @ input_vector.T  # (16, 4)
    d_loss__d_b_first_hidden = d_loss__d_first_algebra  # (16, 1)

    return (d_loss__d_w_output, d_loss__d_b_output,
            d_loss__d_w_second_hidden, d_loss__d_b_second_hidden,
            d_loss__d_w_first_hidden, d_loss__d_b_first_hidden)

## **Training**

Now that we have defined both the forward and backward propagation, we can start training the network using stochastic gradient descent (SGT).

In [13]:
epochs = 100
learning_rate = .01

for epoch in range(epochs):
    for i in range(len(training_predictors)):
        
        # get a batch 
        predictor_batch = training_predictors[i].reshape(-1, 1)
        target_batch = training_response[i]

        # Run the forward propagation
        first_algebra, first_function, second_algebra, second_function, output_algebra, result = forward_propagation(predictor_batch)

        # Run the backward propagation
        d_loss__d_w_output, d_loss__d_b_output, d_loss__d_w_second_hidden, d_loss__d_b_second_hidden, d_loss__d_w_first_hidden, d_loss__d_b_first_hidden =  backward_propagation(first_algebra, first_function, second_algebra, second_function, output_algebra, result, predictor_batch, target_batch)
        
        # update weights and biases
        w_first_hidden -= learning_rate * d_loss__d_w_first_hidden
        b_first_hidden -= learning_rate * d_loss__d_b_first_hidden
        
        w_second_hidden -= learning_rate * d_loss__d_w_second_hidden
        b_second_hidden -= learning_rate * d_loss__d_b_second_hidden
        
        w_output -= learning_rate * d_loss__d_w_output
        b_output -= learning_rate * d_loss__d_b_output

## Testing

Finally, as the weights and biases have been optimised, we can test how well does the model perform on data it hasn't seen before: The testing split we made at the start. We can get the confusion matrix and the precision like so:

In [14]:
predictions = forward_propagation(testing_predictors.T)[5].round().astype(int)

TP = FP = TN = FN = 0

for actual, predicted in zip(testing_response.flatten().tolist(), predictions.flatten().tolist()):
    if actual == 1 and predicted == 1:
        TP += 1
    elif actual == 0 and predicted == 0:
        TN += 1
    elif actual == 0 and predicted == 1:
        FP += 1
    elif actual == 1 and predicted == 0:
        FN += 1

print("Confusion Matrix:")
print(f"          Predicted 0    Predicted 1")
print(f"Actual 0     {TN}             {FP}")
print(f"Actual 1     {FN}             {TP}")
print("")

accuracy = (predictions == testing_response.T).mean() 
print("Accuracy: ", accuracy * 100, "%")

Confusion Matrix:
          Predicted 0    Predicted 1
Actual 0     266             0
Actual 1     1             190

Accuracy:  99.78118161925602 %


That is, the model correctly analysed nearly all of the bank notes, except for one. However, this should be taken with a grain of salt.

# **UNDER ABSOLUTELY NO CIRCUMSTANCES SHOULD THIS MODEL BE USED IN PRACTICE**. 

Speaking a bit more directly, **this is just a cool maths demonstration using a random dataset I found online, NOT an actual program to detect forgery**. 

- There are way better ways to create neural networks. (Like Torch or TensorFlow...)
- I did not independently verify the data collection and processing methodology.
- What currency are even the bank notes in the data? I don't think the publication actually says that. 
- The dataset is quite small.
- A plethora of things are likely to go wrong if applied to real world problems.

**So please, don't actually use the model out there. It was never meant for the real world.**